In [1]:
import sys
sys.path.insert(0, '../')

from src.time_tagger import TimeTaggerController, TT_Simulator, two_particle_states
from src.kinetic_mount_controller import KineticMountControl
from src.osc import OSCCommunicator, OSCTarget
import time
import threading
import random

In [2]:
# only run if not already created
if 'KMC' not in globals():  
    KMC = KineticMountControl(number_of_devices=2, address_search_depth=0)
    #KMC.swap_alice_bob()

TTC = TimeTaggerController()
# connect KMC and TTC
TTC.setKineticMountController(KMC)
# set alice channels
TTC.set_alice_reflection_channel(1)
TTC.set_alice_transmission_channel(2)
# set bob channels
TTC.set_bob_reflection_channel(4)
TTC.set_bob_transmission_channel(3)


# set bob home offset
KMC.bob.set_home_offset(-43 % 360)
KMC.bob.home()
# set alice home offset
KMC.alice.set_home_offset(9)
KMC.alice.home()




ASSIGNING CONTROLLERS
---------------------------------------------------------------
Ports Found: ['COM8', 'COM9']
---------------------------------------------------------------
Device succesfully connected (Rotation Mount) (Address: 0)
Device succesfully connected (Rotation Mount) (Address: 0)


In [3]:
# Global flag to control the loop
resultTranslate = {0 : [1, 1],
                   1 : [1, -1],
                   2 : [-1, 1],
                   3 : [-1, -1]}
# a A b B
Q_angles = [0, -22.5, 11.5, -11.5]
C_angles = [0, 22.5, -45, -22.5]

                    # ab, aB, Ab, AB
angle_pair_dict = {'Q_all':[(0, 11.5), (0, -11.5), (-22.5, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'Q_diagonal': [(0, 11.5), (-22.5, -11.5)],
                    # ab, AB
                   'C_diagonal': [(0, -45), (22.5, -22.5)],
                    # AB
                   'C_single': [(-45, -22.5)],
                   }

# for this perf we only want Q_all
angle_pairs = angle_pair_dict['Q_all']

# setup system with correct initial angles
KMC.rotate_simulataneously(*angle_pairs[0])

def performance_cycle(cycles):
    """The function you want to execute repeatedly."""
   
    time_of_last_visual = time.time()
    prev_theta_a, prev_theta_b = angle_pairs[0]
    for _ in range(cycles):
        theta_a, theta_b = random.choice(angle_pairs)
        
        measurement_results, prev_theta_a, prev_theta_b = TTC.get_single_measurement_metronome(angle_pairs, theta_a, theta_b, prev_theta_a, prev_theta_b, integration_time=0.05, max_time=0.1, min_event_count=1)
        

In [4]:
performance_cycle(100)

Collected 68 events in 0.06 seconds.
Timing Summary: {'createCoincidenceChannels': 0.0, 'rotate_simultaneously': 0.36505866050720215, 'stream_loop': 0.06405091285705566, 'total': 0.4291095733642578}
Collected 42 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.3130002021789551, 'stream_loop': 0.06260561943054199, 'total': 0.37560582160949707}
Collected 32 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.312056303024292, 'stream_loop': 0.06273412704467773, 'total': 0.3747904300689697}
Collected 27 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.30901575088500977, 'stream_loop': 0.06224870681762695, 'total': 0.3712644577026367}
Collected 40 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.31049084663391113, 'stream_loop': 0.06108832359313965, 'total': 0.3715791702270508}
Collected 26 events in 0.06 seconds.
Timing Summary: {'rotate_simultaneously': 0.311908483505249, 'stream_loop': 0.06236433982849121, 'total': 

KeyboardInterrupt: 